In [169]:
import pandas as pd
import sklearn as sk
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import svm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 25)
Df = pd.read_csv("12Buckets.csv")

In [268]:
Groups = ['Safeties','Corners','InsideBackers','Edge','InteriorDLine','WRs','TEs','InteriorOLine','Tackles','QB','RB','Special']
Df.head()

,Year,Team,Safeties,Corners,InsideBackers,Edge,InteriorDLine,WRs,TEs,InteriorOLine,Tackles,QB,RB,Special,Dead,IR,Prac,Susp,Wins,Playoff Wins,SuperBowl Win,Dist
0,2005,cardinals,1.402731,-0.207893,-0.704751,0.637538,-0.524513,0.666781,-0.808983,3.703935,0.513275,-0.200323,-1.050627,1.269564,8.08,0.16,0.09,0.0,5,0,0,5.484357
1,2006,cardinals,1.968584,-0.894881,-0.620196,1.020491,-0.200519,0.858693,-0.797375,4.104730,-0.310605,-0.153475,2.053163,0.192998,1.18,0.61,0.15,0.0,5,0,0,5.192938
2,2007,cardinals,1.927200,-1.231017,-0.680460,0.750435,-0.728735,0.483166,-0.943145,1.903609,0.425250,0.211767,1.018352,-0.611292,2.07,13.01,0.08,0.0,8,0,0,4.273319
3,2008,cardinals,1.401572,-1.141278,0.977356,-0.113338,-0.431277,0.562258,-0.953613,1.769090,2.042454,-0.268646,0.866360,-1.143908,4.09,0.23,0.32,0.0,9,3,0,4.817919
4,2009,cardinals,1.503735,0.174713,1.021642,-0.924559,0.278701,1.189041,-1.609632,0.775470,0.265150,0.469486,-0.344347,-0.131045,6.92,5.68,0.21,0.0,10,1,0,4.416676


In [248]:
def Normalize(x):
    RetList = []
    for position in Groups:
        dfyear = Df[Df['Year'] == x['Year']]
        val = (x[position] - np.mean(dfyear[position]))/(np.std(dfyear[position]))
        
        RetList.append(val)
    
    return pd.Series(RetList)
    
    
    
Df[Groups] = Df.apply(Normalize, axis = 1)

In [249]:
Team = 'texans'
Year = 2018

In [250]:


def DistAllTeams(x):
    TotD = 0
    for Position in Groups:
        if ((x['Team'] == Team)|(x['Year'] >= Year)):
            return 100
        Di = (x[Position]-TheTeam[Position])**2
        TotD += Di
    Dist = np.sqrt(TotD)
    return Dist
    
    
    
    
## Specific Team's Distances    
TheTeam = Df[(Df['Team'] == Team)&(Df['Year'] == Year)]

Df['Dist'] = Df.apply(DistAllTeams, axis = 1)

In [251]:
Df.sort_values(by = "Dist")[:10].head()

,Year,Team,Safeties,Corners,InsideBackers,Edge,InteriorDLine,WRs,TEs,InteriorOLine,Tackles,QB,RB,Special,Dead,IR,Prac,Susp,Wins,Playoff Wins,SuperBowl Win,Dist
147,2017,broncos,-0.275769,1.275990,0.569405,1.748226,-1.193978,1.185370,-0.934975,-0.048269,-0.582677,-1.713965,-0.287445,-0.674031,0.28,17.12,0.00,0.0,5,0,0,2.581396
36,2011,ravens,1.087588,0.674492,0.425132,0.578847,0.365045,0.255663,-1.050098,1.027127,-0.969615,-0.756150,-0.408399,0.412685,3.14,6.98,0.56,0.0,12,2,0,3.152850
392,2007,steelers,0.490220,0.213366,2.186756,0.718514,-0.170756,0.386065,0.562288,1.654959,-0.788202,-0.729340,-0.187249,-0.848400,3.09,10.45,0.07,0.0,10,0,0,3.158269
146,2016,broncos,0.597044,1.330626,0.057845,1.680226,-1.157151,1.450945,0.149442,-1.443006,0.322534,-1.502441,0.639666,-0.710087,9.82,12.54,0.00,0.0,9,0,0,3.257890
35,2010,ravens,0.704172,0.240602,0.103506,0.276714,-0.465091,-0.096207,0.665049,0.002709,-0.391633,-1.002982,-0.014877,0.481461,0.27,6.33,0.33,0.0,12,1,0,3.309145


The teams most similar to the 2018 Texans (that aren't the Texans or in the future) are the 2017 Broncos (5 Wins), the 2011 Ravens (12 Wins), and the 2007 Steelers (10 Wins). 

### 2017 Broncos (5 Wins)

In [277]:
TopTeams = Df.sort_values(by = "Dist")[:1]

NextYear = Df[(Df['Team'] == TopTeams.iloc[0,1])&(Df['Year'] == TopTeams.iloc[0,0]+1)]

Result = NextYear.copy()

Groups2 = Groups.copy()
Groups2.extend(['Dead','IR','Prac','Susp','Wins','Playoff Wins'])
for i in range(0,len(Groups2)):
    Position = Groups2[i]
    NewVal = NextYear[Position].values - TopTeams[Position].values
    Result.iloc[0,i+2] = NewVal
    
Result.iloc[0,0] = '2017 to 2018'
Result

,Year,Team,Safeties,Corners,InsideBackers,Edge,InteriorDLine,WRs,TEs,InteriorOLine,Tackles,QB,RB,Special,Dead,IR,Prac,Susp,Wins,Playoff Wins,SuperBowl Win,Dist
148,2017 to 2018,broncos,-0.227726,-0.221059,0.485421,-0.923957,0.176749,-1.961223,-0.427315,0.270024,-0.233475,1.210933,-1.335786,0.568758,0.06,3.84,0.0,0.0,1,0,0,100.0


#### The 2018 Broncos spent considerably less on WRs, Edge Rushers, and RBs, while spending considerably more on their QB, which ultimately only led to a 1-win improvement

In [283]:
TopTeams = Df.sort_values(by = "Dist")[1:2]

NextYear = Df[(Df['Team'] == TopTeams.iloc[0,1])&(Df['Year'] == TopTeams.iloc[0,0]+1)]

Result2 = NextYear.copy()

for i in range(0,len(Groups2)):
    Position = Groups2[i]
    NewVal = NextYear[Position].values - TopTeams[Position].values
    Result2.iloc[0,i+2] = NewVal
    
Result2.iloc[0,0] = str(TopTeams['Year'].values[0]) + " to " +  str(NextYear['Year'].values[0])
Result2

,Year,Team,Safeties,Corners,InsideBackers,Edge,InteriorDLine,WRs,TEs,InteriorOLine,Tackles,QB,RB,Special,Dead,IR,Prac,Susp,Wins,Playoff Wins,SuperBowl Win,Dist
37,2011 to 2012,ravens,0.500458,-0.311136,0.253507,-0.906201,0.684663,-0.264289,-0.602289,-1.629572,0.274676,0.374219,1.181989,-0.764452,6.47,6.74,-0.18,0.0,-2,2,1,3.949323


#### The 2012 Ravens spend much less on Interior Offensive Line and Edge and much more on RB and Interior Defensive Line. The 2012 offseason saw the Ravens lose G Ben Grubbs and DE/NT Cory Redding and drafting future all-pros in Courtney Upshaw and Kelechi Osemele. The Ravens also re-signed RB Ray Rice to a large deal and Interior D-Lineman Haloti Ngata's contract value increased significantly. This ultimately led to fewer regular season wins, but a Super Bowl Win